stackingのアンサンブル用シート

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import scipy as sp
import lightgbm as lgb
import category_encoders as ce
# import mojimoji
import re
from cmath import nan
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
import optuna
from sklearn.metrics import mean_squared_error
import japanize_matplotlib

In [2]:
###訓練データの読み込み
house_age = pd.read_csv('house_age.csv')
area_size = pd.read_csv('area_size.csv')
room_arrange = pd.read_csv('room_arrange.csv')
contract_span = pd.read_csv('contract_span.csv')#欠損値をゼロ埋めした契約期間
contract_span2 = pd.read_csv('contract_span2.csv')#欠損値を欠損値のままにした契約期間
reg_rent = pd.read_csv('reg_rent.csv')
floor_scores = pd.read_csv('floor_scores.csv')#「所在階」と「全体の階数」が連動して欠損値になる場合のある「所在階」
floor_scores2 = pd.read_csv('floor_scores2.csv')#「所在階」と「全体の階数」が独立している「所在階」
Floor_scores = pd.read_csv('capital_floor_scores.csv')#「所在階」と「全体の階数」が連動して欠損値になる場合のある「全体の階数」
Floor_scores2 = pd.read_csv('capital_floor_scores2.csv')#「所在階」と「全体の階数」が独立している「全体の階数」
stations = pd.read_csv('stations.csv')
minits = pd.read_csv('minits.csv')
addresses = pd.read_csv('addresses.csv')
room_arrange = pd.read_csv('room_arrange.csv')
buildings = pd.read_csv('buildings.csv')

rent = pd.read_csv('rent.csv')


##テストデータの読み込み
test_house_age = pd.read_csv('test_house_age.csv')
test_area_size = pd.read_csv('test_area_size.csv')
test_room_arrange = pd.read_csv('test_room_arrange.csv')
test_contract_span = pd.read_csv('test_contract_span.csv')
test_contract_span2 = pd.read_csv('test_contract_span2.csv')
test_reg_rent = pd.read_csv('test_reg_rent.csv')
test_floor_scores = pd.read_csv('test_floor_scores.csv')
test_floor_scores2 = pd.read_csv('test_floor_scores2.csv')
test_Floor_scores = pd.read_csv('test_capital_floor_scores.csv')
test_Floor_scores2 = pd.read_csv('test_capital_floor_scores2.csv')
test_stations = pd.read_csv('test_stations.csv')
test_minits = pd.read_csv('test_minits.csv')
test_addresses = pd.read_csv('test_addresses.csv')
test_room_arrange = pd.read_csv('test_room_arrange.csv')
test_buildings = pd.read_csv('test_buildings.csv')

lightGBMとDARTとGOSSでstackingのアンサンブルを行う。

In [15]:
X_train = pd.concat([house_age, area_size, contract_span, reg_rent, floor_scores2, Floor_scores2, stations, minits, addresses, room_arrange[["部屋数","L","D","K","S"]]], axis=1)
y_train = rent

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_test = pd.concat([test_house_age, test_area_size, test_contract_span, test_reg_rent, test_floor_scores2, test_Floor_scores2, test_stations, test_minits, test_addresses, test_room_arrange[["部屋数","L","D","K","S"]]], axis=1)

category_lists = ['定期借家','最寄り駅', '所在地', 'L', 'D','K','S']
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

params = {
    'objective':'regression',
    'metrics':'rmse',
    'reg_lambda': 4.430375245218262e-06,
    'max_bin': 502,
    'num_leaves': 97
}

model = lgb.LGBMRegressor(
                    params
                    )
# model = lgb.train(
#                     params,
#                     lgb_train, 
#                     valid_sets=[lgb_train, lgb_eval], 
#                     verbose_eval=10, 
#                     num_boost_round=3000, 
#                     early_stopping_rounds=10,
#                     categorical_feature = category_lists
#                     )

# y_pred = model.predict(X_test, num_iteration=model.best_iteration)

# # feature importanceを表示
# importance = pd.DataFrame(model.feature_importance(importance_type = "gain"), index=X_train.columns, columns=['importance'])
# display(importance)


In [16]:
X_train = pd.concat([house_age, area_size, contract_span, reg_rent, floor_scores2, Floor_scores2, stations, minits, addresses, room_arrange[["部屋数","L","D","K","S"]]], axis=1)
y_train = rent
X_test = pd.concat([test_house_age, test_area_size, test_contract_span2, test_reg_rent, test_floor_scores2, test_Floor_scores2, test_stations, test_minits, test_addresses, test_room_arrange[["部屋数","L","D","K","S"]]], axis=1)

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [18]:
from sklearn.linear_model import LinearRegression #重回帰分析

category_lists = ['定期借家','最寄り駅', '所在地', 'L', 'D','K','S']
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

params_1 = {
    'objective':'regression',
    'metrics':'rmse',
    'reg_lambda': 4.430375245218262e-06,
    'max_bin': 502,
    'num_leaves': 97
}

# モデルのインスタンス
model_1 = lgb.LGBMRegressor(
                    params_1
                    )
# model_1 = lgb.train(
#                     params,
#                     lgb_train, 
#                     valid_sets=[lgb_train, lgb_eval], 
#                     verbose_eval=10, 
#                     num_boost_round=1000, 
#                     early_stopping_rounds=10,
#                     categorical_feature = category_lists
#                     )

# モデルのインスタンス
category_lists = ['定期借家','最寄り駅', '所在地', 'L', 'D', 'K','S']
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

params_2 = {
    'objective':'regression',
    'boosting_type':'dart',
    'metrics':'rmse',
    'reg_lambda': 2.1600820741402023e-05,
    'max_bin': 556,
    'num_leaves': 126
}

model_2 = lgb.LGBMRegressor(
                    params_2
                    )
# model_2 = lgb.train(
#                     params,
#                     lgb_train, 
#                     valid_sets=[lgb_train, lgb_eval], 
#                     verbose_eval=10, 
#                     num_boost_round=1000, 
#                     early_stopping_rounds=10,
#                     categorical_feature = category_lists
#                     )

# モデルのインスタンス
category_lists = ['定期借家', '最寄り駅', '所在地', 'L', 'D', 'K','S']
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

params_3 = {
    'objective':'regression',
    'boosting_type':'goss',
    'metrics':'rmse',
    'reg_lambda': 1.2667345328336822e-06,
    'max_bin': 405,
    'num_leaves': 118
}
# params = {
#     'objective':'regression',
#     'boosting_type':'goss',
#     'metrics':'rmse',
#     'reg_lambda': 1.2667345328336822e-06,
#     'max_bin': 405,
#     'num_leaves': 118
# }

model_3 = lgb.LGBMRegressor(
                    params_3
                    )

# モデルの学習
model_1.fit(X_train, y_train)
model_2.fit(X_train, y_train)
model_3.fit(X_train, y_train)

# 予測値の作成
pred_1 = model_1.predict(X_test)
pred_2 = model_2.predict(X_test)
pred_3 = model_3.predict(X_test)


TypeError: Unknown type of parameter:boosting_type, got:dict

In [12]:
# 第一段階の予測値
first_pred_1 = model_1.predict(X_valid)
first_pred_2 = model_2.predict(X_valid)
first_pred_3 = model_3.predict(X_valid)

# 第一段階の予測値をまとめる（メタモデルの特徴量）
stack_pred = np.column_stack((first_pred_1,first_pred_2,first_pred_3))

# メタモデルの作成
meta_model = LinearRegression()
# 第一段階の予測値の答え = y_valid
meta_model.fit(stack_pred, y_valid)

# 事前に予測しておいた値でスタッキングの精度を確認する
stack_test_pred = np.column_stack((pred_1, pred_2, pred_3))
meta_test_pred = meta_model.predict(stack_test_pred)
#print ("メタモデルの平均2乗誤差: {:.4f}".format(mean_squared_error(y_test, meta_test_pred)))